In [2]:
import preprocess as pp
import configs
import persister
import relation
import lda

from stanfordcorenlp import StanfordCoreNLP
from nltk import sent_tokenize, word_tokenize
from nltk.tree import Tree

In [ ]:
# 文档向量化测试
# count vectorizer

In [ ]:
a = "Many existing knowledge bases(KBs), including Freebase, Yago, and NELL, rely on a ﬁxed ontology, given as an input to the system, which deﬁnes the data to be cataloged in the KB, i.e., a hierarchy of categories and relations between them. The system then extracts facts that match the predeﬁned ontology. We propose an unsupervised model that jointly learns a latent ontological structure of an input corpus, and identiﬁes facts from the corpus that match the learned structure. Our approach combines mixed membership stochastic block models and topic models to infer a structure by jointly modeling text, a latent concept hierarchy, and latent semantic relationships among the entities mentioned in the text. As a case study, we apply the model to a corpus of Web documents from the software domain,and evaluate the accuracy of the various components of the learned ontology. "
b = ["hello world. what the fuck."]

In [ ]:
c = [" ".join(preprocess(a))]
vector = CountVectorizer(ngram_range=(1, 2), vocabulary=["knowledge base"], stop_words='english')
vector.build_analyzer()
x = vector.fit_transform(c)

In [ ]:
estimator = CountVectorizer(ngram_range=(1, 2), vocabulary=['fuck', 'hello', 'world', "hello world"], stop_words='english')
estimator.build_analyzer()
res = estimator.fit_transform(b)

In [ ]:
estimator.get_feature_names()

In [ ]:
res.toarray()
# estimator.vocabulary_

In [ ]:
# 斯坦福nlp工具测试
# stanfordcorenlp

In [ ]:
nlp = StanfordCoreNLP('http://localhost', port=9000)
sentence = 'Guangdong University of Foreign Studies is located in Guangzhou.'
print('Tokenize:', nlp.word_tokenize(sentence))
print('Part of Speech:', nlp.pos_tag(sentence))
print('Named Entities:', nlp.ner(sentence))
print('Constituency Parsing:', nlp.parse(sentence))
print('Dependency Parsing:', nlp.dependency_parse(sentence))

#nlp.close() # Do not forget to close! The backend server will consume a lot memery.

In [ ]:
# 官方corenlp api
# stanza

In [ ]:
# 新闻数据集测试
# 20newsgroups

In [ ]:
import newsgroups
size = 500
tmp = newsgroups.get_news_data(size)
newsdata = []
for cate in tmp:
    newsdata.extend(cate)

In [ ]:
texts = [' '.join(pp.preprocess_abstract(a)) for a in tmp]
persister.save_json(configs.NEWSDATA, texts)

In [ ]:
# 处理成句子
# nltk.sent_tokenize

In [ ]:
newssent = []
for news in newsdata:
    newssent.append(pp.split2sent(news))
persister.save_json(configs.NEWSSENT, newssent)

In [3]:
newssenttoken = []
for news in newssent:
    tokenized_lemmatized_news = []
    for sent in news:
        tokenized_lemmatized_news.append(relation.lemmatize_sent_words(sent))
    newssenttoken.append(tokenized_lemmatized_news)
persister.save_json(configs.NEWSSENTTOKEN, newssenttoken)

In [ ]:
# load preprossed data

In [4]:
texts = persister.load_json(configs.NEWSDATA)
newssent = persister.load_json(configs.NEWSSENT)
newssenttoken = persister.load_json(configs.NEWSSENTTOKEN)

In [ ]:
# load lda res
terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA)
lda.print_topics(topic_word, terms, doc_topic)

In [11]:
relation.CLI.pos_tag(newssent[432][13])

[('3D', 'NN'), ('graphics', 'NNS'), ('editors', 'NNS'), ('a.', 'IN')]

In [12]:
dparse = relation.CLI.dependency_parse(newssent[432][13])

In [16]:
from nltk.tokenize.stanford import StanfordTokenizer
er=StanfordTokenizer()

LookupError: 

===========================================================================
  NLTK was unable to find stanford-postagger.jar! Set the CLASSPATH
  environment variable.

  For more information, on stanford-postagger.jar, see:
    <http://nlp.stanford.edu/software/tokenizer.shtml>
===========================================================================

In [9]:
cparse = relation.CLI.parse(newssent[432][13])
print(cparse)
t = Tree.fromstring(cparse)
t.draw()

(ROOT
  (NP
    (NP (NN 3D) (NNS graphics))
    (NP
      (NP (NNS editors))
      (ADJP (IN a.)))))


In [15]:
t.triples()

AttributeError: 'Tree' object has no attribute 'triples'

In [5]:
s = relation.get_sent_idx(newssenttoken[432], "graphic")

In [7]:
s

{13,
 26,
 34,
 38,
 93,
 119,
 125,
 133,
 159,
 178,
 213,
 214,
 226,
 231,
 237,
 243,
 284,
 290,
 302,
 314,
 333,
 362,
 371,
 374,
 383}

In [12]:
newssenttoken[432][13]

['3D', 'graphic', 'editor', 'a.']